In [8]:
import pandas as pd
import numpy as np
import plotly.express as px
import scipy.stats as stats
import statsmodels.stats.api as sms
from math import ceil

In [16]:
df = pd.read_csv('ab_data.csv')
df.head()

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


In [7]:
df.describe(include = 'all')

,user_id,timestamp,group,landing_page,converted
count,294478.000000,294478,294478,294478,294478.000000
unique,NaN,294478,2,2,NaN
top,NaN,2017-01-14 13:09:41.609677,treatment,old_page,NaN
freq,NaN,1,147276,147239,NaN
mean,787974.124733,NaN,NaN,NaN,0.119659
std,91210.823776,NaN,NaN,NaN,0.324563
min,630000.000000,NaN,NaN,NaN,0.000000
25%,709032.250000,NaN,NaN,NaN,0.000000
50%,787933.500000,NaN,NaN,NaN,0.000000
75%,866911.750000,NaN,NaN,NaN,0.000000


Definindo um tamanho amostral que nos garanta significância estatística nos testes:

In [15]:
effect_size = sms.proportion_effectsize (0.13,0.15)

required_n = sms.NormalIndPower().solve_power(
    effect_size,
    power = 0.9,
    alpha = 0.05,
    ratio =1)

required_n = ceil(required_n)

print (f'São necessárias {required_n} observações para o teste!')

São necessárias 6319 observações para o teste!


Limpando o dataset: Temos que ter certeza de contabilizar clientes *únicos* na análise para que não haja interferência nos testes:

In [ ]:
count = df['user_id'].value_counts(ascending = False)
multi_count = count[count > 1].count()

print (f'Existem {multi_count} usuários que realizaram mais de um acesso')

Removendo estes usuários:

In [19]:
users_to_delete = count[count > 1].index

df = df[~ df['user_id'].isin(users_to_delete)]

print (f'O dataset atualizado agora possui {df.shape[0]} observações')

O dataset atualizado agora possui 286690 observações


Feita a limpeza, faremos a seleção das amostras:

In [21]:
control_group = df[df['group'] == 'control'].sample(n = required_n)

treatment_group = df[df['group'] == 'treatment'].sample(n = required_n)

test = pd.concat([control_group, treatment_group], axis = 0)
test = test.reset_index(drop = True)

test['group'].value_counts()

treatment    6319
control      6319
Name: group, dtype: int64

In [35]:
conversion_rate = test.groupby(by = 'group')['converted']

standard_deviation = lambda x: np.std(x)

standard_error = lambda x: stats.sem(x)

conversion_rate = conversion_rate.agg([np.mean, standard_deviation, standard_error])
conversion_rate.columns = ['conversion_rate', 'standard_deviation', 'standard_error']

conversion_rate.style.format('{:3f}')
conversion_rate

,conversion_rate,standard_deviation,standard_error
group,,,
control,0.116632,0.320982,0.004038
treatment,0.120430,0.325464,0.004095


In [45]:
text = conversion_rate['conversion_rate'][:]

fig = px.bar(conversion_rate,
            template = 'plotly_white',
            text = text)
fig.show()